<a href="https://colab.research.google.com/github/andersonfaller/detoxlm/blob/main/Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install accelerate
!pip install bitsandbytes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import collections
import itertools
import functools
import math
import numpy as np
import os
import random
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
import pandas as pd
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader, Dataset
from tqdm.notebook import tqdm
from typing import List, Type

from googleapiclient import discovery
import json

In [3]:
!wget nc https://github.com/andersonfaller/detoxlm/raw/main/prompts.7z
!7z e prompts.7z

--2022-11-24 03:23:40--  http://nc/
Resolving nc (nc)... failed: No address associated with hostname.
wget: unable to resolve host address ‘nc’
--2022-11-24 03:23:40--  https://github.com/andersonfaller/detoxlm/raw/main/prompts.7z
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/andersonfaller/detoxlm/main/prompts.7z [following]
--2022-11-24 03:23:40--  https://raw.githubusercontent.com/andersonfaller/detoxlm/main/prompts.7z
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15732789 (15M) [application/octet-stream]
Saving to: ‘prompts.7z.1’

prompts.7z.1        100%[===================>]  15.00

In [4]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


In [ ]:
API_KEY = input()

In [5]:
from google.colab import drive
import pickle

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# Para desenvolvimento, sugerimos usar o GPT menor, com 1.3B parametros.
#model = transformers.GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B", device_map="auto", load_in_8bit=True)
#tokenizer = transformers.AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")

# Se estiver usando o colab pro, conseguimos rodar um GPT-J de 6B de parametros.
model = transformers.GPTJForCausalLM.from_pretrained("EleutherAI/gpt-j-6B", revision="float16", torch_dtype=torch.float16, low_cpu_mem_usage=True)
tokenizer = transformers.AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")
model = model.to(device)

In [8]:
bert_tokenizer = transformers.BertTokenizer.from_pretrained("bert-base-cased")
bert_model = transformers.BertForSequenceClassification.from_pretrained("bert-base-cased").to(device)
bert_model = bert_model.to(device)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [9]:
filename = '/content/drive/MyDrive/' + 'bert_perspective.pt'
bert_model.load_state_dict(pickle.load(open(filename, 'rb')))

<All keys matched successfully>

In [10]:
prompt = (
    "This bastard came to me asking for something I couldn't"
    )

tokens = tokenizer(prompt, return_tensors="pt")

gen_tokens = model.generate(
    inputs = tokens.input_ids.to(device),
    attention_mask = tokens.attention_mask.to(device),
    pad_token_id = tokenizer.eos_token_id,
    do_sample = True,
    temperature = 0.9,
    max_length = len(tokens.input_ids[0]) + 20,
    num_return_sequences = 10
)

gen_texts = tokenizer.batch_decode(gen_tokens)
continuations = [text[len(prompt):].strip() for text in gen_texts]

In [10]:
client = discovery.build(
  "commentanalyzer",
  "v1alpha1",
  developerKey=API_KEY,
  discoveryServiceUrl="https://commentanalyzer.googleapis.com/$discovery/rest?version=v1alpha1",
  #static_discovery=False,
)

def analyse_toxicity_bert(texts):
  inputs = bert_tokenizer.batch_encode_plus(texts, max_length = 50, padding = True, truncation = True, return_tensors = "pt")
  inputs = inputs.to(device)
  with torch.no_grad():
    logits = bert_model(**inputs)
  probs = torch.softmax(logits.logits, dim = -1)
  return probs[:,0].cpu().numpy()


def analyse_toxicity(text):
  analyze_request = {
    'comment': { 'text': text },
    'requestedAttributes': {'TOXICITY': {}}
  }
  try:
    response = client.comments().analyze(body=analyze_request).execute()
    toxicity = response['attributeScores']['TOXICITY']['summaryScore']['value']
  except:
    print('ERRO')
    return np.nan
  return toxicity

In [12]:
analyse_toxicity_bert(['Fuck you', 'Hey, handsome'])

array([0.9600472 , 0.07353213], dtype=float32)

In [13]:
def my_multinomial(weights):
   total = sum(weights)
   r = random.uniform(0, total)
   posicao = 0
   for i, w in enumerate(weights):
      if posicao + w >= r:
         return i
      posicao += w

def decode_sampling(logits, temperature = 1):
  probs = torch.softmax(logits/temperature, dim = -1)
  return my_multinomial(probs)

def decode_top_k(logits, temperature = 1, k = 1):
  probs = torch.softmax(logits/temperature, dim = -1)
  top_k = probs.topk(k)
  return top_k.indices[my_multinomial(top_k.values)].item()

def decode_top_p(logits, temperature = 1, p = 1):
  probs = torch.softmax(logits/temperature, dim = -1)
  top_p_idx = []
  top_p_values = []
  idx = torch.argsort(logits, descending = True)
  for i in idx:
    top_p_idx.append(i)
    top_p_values.append(probs[i])
    if sum(top_p_values) >= p:
      break
  return top_p_idx[my_multinomial(top_p_values)].item()

def generate(model, prompt, max_length, temperature = 1, top_k = -1, top_p = -1):
  if top_k > 0:
    decoder = lambda x: decode_top_k(x, temperature, top_k)
  elif top_p > 0:
    decoder = lambda x: decode_top_p(x, temperature, top_p)
  else:
    decoder = lambda x: decode_sampling(x, temperature)
  
  input_ids = tokenizer(prompt, return_tensors="pt").input_ids[0]

  for i in range(max_length):
    with torch.no_grad():
      logits = model(input_ids.cuda()).logits[-1] # último token
    next_token = decoder(logits)
    input_ids = torch.cat([input_ids, torch.LongTensor([next_token])])
  
  return tokenizer.decode(input_ids)

def generate_modified(model, prompt, max_length, temperature = 1, top_k = -1, top_p = -1):
  if top_k > 0:
    decoder = lambda x: decode_top_k(x, temperature, top_k)
  elif top_p > 0:
    decoder = lambda x: decode_top_p(x, temperature, top_p)
  else:
    decoder = lambda x: decode_sampling(x, temperature)
  
  input_ids = tokenizer(prompt, return_tensors="pt").input_ids[0]

  for i in range(max_length):
    with torch.no_grad():
      logits = model(input_ids.cuda()).logits[-1] # último token
    next_token = decoder(logits)
    input_ids = torch.cat([input_ids, torch.LongTensor([next_token])])
  
  return tokenizer.decode(input_ids)

In [14]:
client = discovery.build(
  "commentanalyzer",
  "v1alpha1",
  developerKey=API_KEY,
  discoveryServiceUrl="https://commentanalyzer.googleapis.com/$discovery/rest?version=v1alpha1",
  #static_discovery=False,
)

In [15]:
with open('prompts.jsonl') as file:
  prompts = [json.loads(line) for line in file.readlines()]

df_prompts = pd.DataFrame.from_records([p['prompt'] for p in prompts])
df_continuations = pd.DataFrame.from_records([p['continuation'] for p in prompts])
series_challenging = pd.DataFrame.from_records(prompts)['challenging']


In [ ]:
#df_my_continuations = pd.DataFrame()
df_my_continuations = pickle.load(open('/content/drive/MyDrive/df_my_continuations_k1.pkl', 'rb'))

In [ ]:
max_length = 20
num_sequences = 1

#for i, row in df_prompts.iterrows():
contador = 1
while True: 
  i = random.choice(df_prompts.index)
  if i in df_my_continuations.index:
    continue
  
  row = df_prompts.loc[i]

  prompt = row['text']
  tokens = tokenizer(prompt, return_tensors="pt")

  gen_tokens = model.generate(
      inputs = tokens.input_ids.to(device),
      attention_mask = tokens.attention_mask.to(device),
      top_k = 1, # remember to change this
      pad_token_id = tokenizer.eos_token_id,
      do_sample = True,
      temperature = 0.9,
      max_length = len(tokens.input_ids[0]) + max_length,
      num_return_sequences = num_sequences
  )

  gen_texts = tokenizer.batch_decode(gen_tokens)
  continuations = [text[len(prompt):].strip() for text in gen_texts]

  prompt_toxicity = analyse_toxicity_bert([prompt])[0]
  #continuations_toxicity = [analyse_toxicity(c) for c in continuations]
  continuations_toxicity = analyse_toxicity_bert(continuations)

  df_my_continuations.loc[i, 'prompt'] = row['text']
  df_my_continuations.loc[i, 'prompt_toxicity'] = prompt_toxicity

  df_my_continuations.loc[i, 'min_continuation_toxicity'] = min(continuations_toxicity)
  df_my_continuations.loc[i, 'argmin_continuation_toxicity'] = np.argmin(continuations_toxicity)

  df_my_continuations.loc[i, 'max_continuation_toxicity'] = max(continuations_toxicity)
  df_my_continuations.loc[i, 'argmax_continuation_toxicity'] = np.argmax(continuations_toxicity)

  for j, continuation, continuation_toxicity in zip(range(num_sequences), continuations, continuations_toxicity):
    df_my_continuations.loc[i, f'continuation_{j}'] = continuation
    df_my_continuations.loc[i, f'continuation_toxicity_{j}'] = continuation_toxicity

  if contador%20 == 0:
    print(f'contador = {contador}; i = {i}')
    pickle.dump(df_my_continuations, open('/content/drive/MyDrive/df_my_continuations_k1.pkl', 'wb'))

  #print(i, prompt_toxicity, continuation_toxicity)
  contador += 1
  if contador > 1000:
    break

pickle.dump(df_my_continuations, open('/content/drive/MyDrive/df_my_continuations_k1.pkl', 'wb'))

contador = 20; i = 85590
contador = 40; i = 47416
contador = 60; i = 44937
contador = 80; i = 86491
contador = 100; i = 8432
contador = 120; i = 76545
contador = 140; i = 66541
contador = 160; i = 99143
contador = 180; i = 34207
contador = 200; i = 34027
contador = 220; i = 59154
contador = 240; i = 94185
contador = 260; i = 54010
contador = 280; i = 66341
contador = 300; i = 17284
contador = 320; i = 61008
contador = 340; i = 98151
contador = 360; i = 86262
contador = 380; i = 63446
contador = 400; i = 22752
contador = 420; i = 47888
contador = 440; i = 77043
contador = 460; i = 95496
contador = 480; i = 17116
contador = 500; i = 59071
contador = 520; i = 28721
contador = 540; i = 30018
contador = 560; i = 66173
contador = 580; i = 28393
contador = 600; i = 62169
contador = 620; i = 36917
contador = 640; i = 54841
contador = 660; i = 63815
contador = 680; i = 16505
contador = 700; i = 71284
contador = 720; i = 87754
contador = 740; i = 30966
contador = 760; i = 43773
contador = 780; i

In [25]:
def generate_controlled_toxicity(prompt, num_tokens, num_sequences = 10, max_step_len = 20, min_toxicity = True):

  while num_tokens > 0:
    max_length = min(num_tokens, max_step_len)
    num_tokens = max(0, num_tokens - max_length)

    tokens = tokenizer(prompt, return_tensors="pt")

    gen_tokens = model.generate(
        inputs = tokens.input_ids.to(device),
        attention_mask = tokens.attention_mask.to(device),
        pad_token_id = tokenizer.eos_token_id,
        do_sample = True,
        temperature = 0.9,
        max_length = len(tokens.input_ids[0]) + max_length,
        num_return_sequences = num_sequences
    )

    gen_texts = tokenizer.batch_decode(gen_tokens)
    continuations = [text[len(prompt):].strip() for text in gen_texts]
    continuations_toxicity = analyse_toxicity_bert(continuations)

    if min_toxicity:
      idx = np.argmin(continuations_toxicity)
    else:
      idx = np.argmax(continuations_toxicity)
    
    prompt = gen_texts[idx]
  
  return prompt

prompt = "The satanic book is on the table"

print(generate_controlled_toxicity(prompt, 60, num_sequences = 15, max_step_len = 30, min_toxicity = True))
print(generate_controlled_toxicity(prompt, 60, num_sequences = 15, max_step_len = 30, min_toxicity = False))

The satanic book is on the table. He wants to know why this book is so important? I reply that it's the only book in the library that tells him the truth.

"It's forbidden, why is it?" He points at the book.

"It's the only one I have, it's the
The satanic book is on the table. There are two other men in the room. One is a young one and the other is an old one. But they are both fat. And it feels like the room is a lot smaller. The young man says to the boys, ‘Hey you fat bastards, get us some water
